In [3]:
import numpy as np

from esm.utils.structure.protein_chain import ProteinChain
from esm.sdk import client
from esm.sdk.api import (
    ESMProtein,
    GenerationConfig,
)

In [11]:
import os
import urllib.request
from Bio.PDB import PDBParser, PDBIO

# Function to fetch and save a PDB file from RCSB

def fetch_pdb(pdb_id, output_dir):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    output_path = os.path.join(output_dir, f"{pdb_id}.pdb")
    try:
        urllib.request.urlretrieve(url, output_path)
        print(f"PDB file {pdb_id} downloaded successfully to {output_path}")
    except Exception as e:
        print(f"Failed to download PDB file {pdb_id}: {e}")
    return output_path

# Specify the PDB ID and output directory
pdb_id = "3OB4"
output_dir = "./pdb_proteins/"

# Fetch the PDB file
pdb_file_path = fetch_pdb(pdb_id, output_dir)

# Load the structure using Bio.PDB
pdb_parser = PDBParser(QUIET=True)
structure = pdb_parser.get_structure("structure", pdb_file_path)

# Iterate over the chains in the structure
for model in structure:
    for chain in model:
        print(f"Chain ID: {chain.id}, Number of residues: {len(chain)}")

PDB file 3OB4 downloaded successfully to ./pdb_proteins/3OB4.pdb
Chain ID: A, Number of residues: 551
Chain ID: B, Number of residues: 3


In [10]:
chain_id = "A"  # Chain ID corresponding to Ara h 2 in the PDB structure
arah2_chain = ProteinChain.from_pdb(pdb_file_path, chain_id)
# Alternatively, we could have used ProteinChain.from_pdb() to load a protein structure from a local PDB file

print(arah2_chain.sequence)

KIEEGKLVIWINGDKGYNGLAEVGKKFEKDTGIKVTVEHPDKLEEKFPQVAATGDGPDIIFWAHDRFGGYAQSGLLAEITPAAAFQDKLYPFTWDAVRYNGKLIAYPIAVEALSLIYNKDLLPNPPKTWEEIPALDKELKAKGKSALMFNLQEPYFTWPLIAADGGYAFKYAAGKYDIKDVGVDNAGAKAGLTFLVDLIKNKHMNADTDYSIAEAAFNKGETAMTINGPWAWSNIDTSAVNYGVTVLPTFKGQPSKPFVGVLSAGINAASPNKELAKEFLENYLLTDEGLEAVNKDKPLGAVALKSYEEELAKDPRIAATMENAQKGEIMPNIPQMSAFWYAVRTAVINAASGRQTVDAALAAAQTNAAARRCQSQLERANLRPCEQHLMQKIQRDEDSRDPYSPSQDPHQERCCNELNEFENNQRCMCEALQQIMENQSDRLQGRQQEQQFKRELRNLPQQCGLRAPQRCDLD


In [8]:
!pip install freesasa

  Using cached freesasa-2.2.1-cp311-cp311-linux_x86_64.whl


In [17]:
from Bio.SeqUtils import seq1
import freesasa
import numpy as np
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster, linkage
import csv

# Extract just the chain of interest from the structure
chain_structure = None
for model in structure:
    chain_structure = model[chain_id]
    break  # Exit after extracting the first model (if multiple models exist)

# Write a temporary PDB file containing only the selected chain
with open("temp_chain.pdb", "w") as temp_pdb:
    io = PDBIO()
    io.set_structure(chain_structure)
    io.save(temp_pdb)

# Initialize FreeSASA structure for the specific chain
freesasa_structure = freesasa.Structure("temp_chain.pdb")

# Run FreeSASA to calculate ASA for each atom
result = freesasa.calc(freesasa_structure)

# Max ASA values for RSA calculation (adjusted per residue type)
max_asa = {
    'A': 113, 'R': 241, 'N': 158, 'D': 151, 'C': 140, 'Q': 189, 'E': 183,
    'G': 85,  'H': 194, 'I': 182, 'L': 180, 'K': 211, 'M': 204, 'F': 218,
    'P': 143, 'S': 122, 'T': 146, 'W': 259, 'Y': 229, 'V': 160
}

# Function to check for N-glycosylation motif (N-X-S/T)
def is_nglycosylated(seq, pos):
    if pos + 2 < len(seq) and seq[pos] == 'N':
        if seq[pos + 2] in ['S', 'T'] and seq[pos + 1] != 'P':
            return 1
    return 0

# Prepare CSV output for residue-level data
with open('asa_rsa_output.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Residue", "ASA", "RSA", "N-Glycosylation"])

    # FreeSASA iterates over atoms, not residues, so we have to match atoms
    atom_idx = 0  # Keep track of FreeSASA atom index

    # Get the sequence of residues for N-glycosylation check
    chain_sequence = [seq1(res.resname) for res in chain_structure.get_residues()]
    print("Chain A Sequence:", "".join(chain_sequence))  # Print sequence for debugging

    # Extract residue information and calculate ASA, RSA, and N-glycosylation
    residue_coords = []  # For storing C-alpha coordinates
    rsa_values = []  # For storing RSA values to calculate averages later
    for i, residue in enumerate(chain_structure.get_residues()):
        try:
            # Filter out non-protein residues (e.g., metals, water)
            if residue.id[0] != ' ':
                continue

            res_id = residue.id[1]  # Residue position in the chain
            amino = residue.resname  # 3-letter amino acid code
            amino_one_letter = seq1(amino)  # Convert to 1-letter code

            # Initialize ASA for the entire residue
            residue_asa = 0.0

            # Iterate over atoms in the residue to sum up their ASA values
            for atom in residue:
                residue_asa += result.atomArea(atom_idx)
                atom_idx += 1  # Move to the next atom

            # Calculate RSA (Relative Solvent Accessibility)
            rsa = residue_asa / max_asa.get(amino_one_letter, 1)
            rsa_values.append(rsa)

            # Check if the residue is N-glycosylated and ensure index is valid
            n_glycosylation = is_nglycosylated(chain_sequence, i) if i + 2 < len(chain_sequence) else 0

            # Create the "Residue" field as position:amino
            residue_field = f"{res_id}:{amino_one_letter}"

            # Write to CSV (position:amino, ASA, RSA, N-Glycosylation)
            writer.writerow([residue_field, residue_asa, rsa, n_glycosylation])

            # Store residue information for clustering
            if residue.has_id("CA"):
                ca_atom = residue["CA"]
                coord = ca_atom.get_coord()
                residue_coords.append((res_id, amino_one_letter, coord))

        except Exception as e:
            print(f"Error processing residue {residue}: {e}")

# Clustering based on 3D proximity
# Extract coordinates for clustering
positions = np.array([coord[2] for coord in residue_coords])

# Calculate pairwise distances between residues based on their Cα coordinates
distance_matrix = pdist(positions)  # Use pdist directly, no need for squareform

# Cluster residues using a hierarchical clustering approach with 8 Å threshold
linkage_matrix = linkage(distance_matrix, method='complete')
distance_threshold = 8.0  # Threshold in Å for clustering
cluster_labels = fcluster(linkage_matrix, t=distance_threshold, criterion='distance')

# Create residue clusters with 1-letter amino acid representation
clusters = {}
for idx, cluster_id in enumerate(cluster_labels):
    if cluster_id not in clusters:
        clusters[cluster_id] = {
            "residues": [],
            "rsa_values": []
        }
    res_id, amino_one_letter = residue_coords[idx][0], residue_coords[idx][1]
    clusters[cluster_id]["residues"].append(f"{res_id}:{amino_one_letter}")
    clusters[cluster_id]["rsa_values"].append(rsa_values[idx])

# Write clusters to a new CSV file for residue clusters with average RSA
with open('residue_clusters.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Residue_Cluster", "Average_RSA"])

    for cluster in clusters.values():
        avg_rsa = sum(cluster["rsa_values"]) / len(cluster["rsa_values"]) if cluster["rsa_values"] else 0
        writer.writerow([";".join(cluster["residues"]), avg_rsa])


Chain A Sequence: KIEEGKLVIWINGDKGYNGLAEVGKKFEKDTGIKVTVEHPDKLEEKFPQVAATGDGPDIIFWAHDRFGGYAQSGLLAEITPAAAFQDKLYPFTWDAVRYNGKLIAYPIAVEALSLIYNKDLLPNPPKTWEEIPALDKELKAKGKSALMFNLQEPYFTWPLIAADGGYAFKYAAGKYDIKDVGVDNAGAKAGLTFLVDLIKNKHMNADTDYSIAEAAFNKGETAMTINGPWAWSNIDTSAVNYGVTVLPTFKGQPSKPFVGVLSAGINAASPNKELAKEFLENYLLTDEGLEAVNKDKPLGAVALKSYEEELAKDPRIAATMENAQKGEIMPNIPQMSAFWYAVRTAVINAASGRQTVDAALAAAQTNAAARRCQSQLERANLRPCEQHLMQKIQRDEDSRDPYSPSQDPHQERCCNELNEFENNQRCMCEALQQIMENQSDRLQGRQQEQQFKRELRNLPQQCGLRAPQRCDLDXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [18]:
# Hydrophilicity scale (Hopp-Woods scale)
hydrophilicity_scale = {
    'A': -0.5, 'R': 3.0, 'N': 0.2, 'D': 3.0, 'C': -1.0,
    'Q': 0.2,  'E': 3.0, 'G': 0.0, 'H': -0.5, 'I': -1.8,
    'L': -1.8, 'K': 3.0, 'M': -1.3, 'F': -2.5, 'P': 0.0,
    'S': 0.3,  'T': -0.4, 'W': -3.4, 'Y': -2.3, 'V': -1.5
}

# Step 1: Update 'asa_rsa_output.csv' with hydrophilicity column
# Re-open the existing CSV and add a hydrophilicity column without duplicating columns
with open('asa_rsa_output.csv', mode='r') as infile:
    reader = csv.reader(infile)
    header = next(reader)  # Skip the header row
    rows = list(reader)

# Check if the header already contains "Hydrophilicity"
if "Hydrophilicity" not in header:
    header.append("Hydrophilicity")

# Calculate and add hydrophilicity values for each row
for row in rows:
    try:
        residue_info = row[0]  # Format is "position:amino"
        position, amino_one_letter = residue_info.split(':')
        # Assign hydrophilicity based on the amino acid using Hopp-Woods scale
        hydrophilicity = hydrophilicity_scale.get(amino_one_letter, 0)
        # If the row already has the hydrophilicity column, update it, else append
        if len(row) < len(header):
            row.append(hydrophilicity)
        else:
            row[header.index("Hydrophilicity")] = hydrophilicity
    except Exception as e:
        print(f"Error processing row {row}: {e}")

# Write the updated data back to 'asa_rsa_output.csv'
with open('asa_rsa_output.csv', mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)  # Write the updated header
    writer.writerows(rows)  # Write all rows with updated hydrophilicity values

# Step 2: Load residue hydrophilicity data from the updated 'asa_rsa_output.csv'
residue_hydrophilicity = {}

# Read the updated residue-level data including hydrophilicity
with open('asa_rsa_output.csv', mode='r') as infile:
    reader = csv.reader(infile)
    header = next(reader)  # Skip the header row
    for row in reader:
        residue_info = row[0]  # Format is "position:amino"
        position, amino = residue_info.split(':')
        position = int(position)  # Convert position to an integer for easier lookup
        hydrophilicity = float(row[header.index("Hydrophilicity")])
        residue_hydrophilicity[position] = hydrophilicity

# Step 3: Update 'residue_clusters.csv' with average hydrophilicity column
clusters_with_hydrophilicity = []

with open('residue_clusters.csv', mode='r') as infile:
    reader = csv.reader(infile)
    header = next(reader)  # Skip the header row

    # Ensure "Average_Hydrophilicity" is in the header
    if "Average_Hydrophilicity" not in header:
        header.append("Average_Hydrophilicity")

    for row in reader:
        residues = row[0].split(';')  # Residues are in the format "position:amino"

        # Calculate the average hydrophilicity for each cluster
        total_hydrophilicity = 0
        residue_count = 0
        for residue in residues:
            position, amino = residue.split(':')
            position = int(position)
            if position in residue_hydrophilicity:
                total_hydrophilicity += residue_hydrophilicity[position]
                residue_count += 1

        # Compute the average hydrophilicity for the cluster
        avg_hydrophilicity = total_hydrophilicity / residue_count if residue_count > 0 else 0

        # Update row with the average hydrophilicity
        if len(row) < len(header):  # If only the cluster information is present
            row.append(avg_hydrophilicity)
        else:
            row[header.index("Average_Hydrophilicity")] = avg_hydrophilicity
        clusters_with_hydrophilicity.append(row)

# Write the updated cluster data back to 'residue_clusters.csv'
with open('residue_clusters.csv', mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)  # Write the updated header
    writer.writerows(clusters_with_hydrophilicity)  # Write all updated rows

In [19]:
# Step 1: Calculate B-factors for residues and update 'asa_rsa_output.csv'
b_factors = []
for residue in chain_structure.get_residues():
    if residue.id[0] == ' ':
        # Calculate the average B-factor for the residue
        avg_b_factor = sum(atom.get_bfactor() for atom in residue) / len(residue)
        b_factors.append(avg_b_factor)

# Update 'asa_rsa_output.csv' to add or update B-factor column
with open('asa_rsa_output.csv', mode='r') as infile:
    reader = csv.reader(infile)
    header = next(reader)
    rows = list(reader)

# Check if the header already contains "B-Factor"
if "B-Factor" not in header:
    header.append("B-Factor")

# Add or update B-factor values for each row
for i, row in enumerate(rows):
    try:
        b_factor = b_factors[i] if i < len(b_factors) else 0
        # If the row already has the B-factor column, update it; otherwise, append
        if len(row) < len(header):
            row.append(b_factor)
        else:
            row[header.index("B-Factor")] = b_factor  # Use index of "B-Factor" in header
    except Exception as e:
        print(f"Error processing row {row}: {e}")

# Write the updated data back to 'asa_rsa_output.csv'
with open('asa_rsa_output.csv', mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)
    writer.writerows(rows)

# Step 2: Load residue B-factor data from 'asa_rsa_output.csv'
residue_b_factor = {}
with open('asa_rsa_output.csv', mode='r') as infile:
    reader = csv.reader(infile)
    header = next(reader)
    for row in reader:
        residue_info = row[0]  # Format is "position:amino"
        position, amino = residue_info.split(':')
        position = int(position)
        b_factor = float(row[header.index("B-Factor")])
        residue_b_factor[position] = b_factor

# Step 3: Update 'residue_clusters.csv' with average B-factor column
clusters_with_b_factors = []
with open('residue_clusters.csv', mode='r') as infile:
    reader = csv.reader(infile)
    header = next(reader)

    # Ensure "Average_B-Factor" is in the header
    if "Average_B-Factor" not in header:
        header.append("Average_B-Factor")

    for row in reader:
        residues = row[0].split(';')  # Residues are in the format "position:amino"

        # Calculate the average B-factor for each cluster
        total_b_factor = 0
        residue_count = 0
        for residue in residues:
            position, amino = residue.split(':')
            position = int(position)
            if position in residue_b_factor:
                total_b_factor += residue_b_factor[position]
                residue_count += 1

        # Compute the average B-factor for the cluster
        avg_b_factor = total_b_factor / residue_count if residue_count > 0 else 0

        # Update row with the average B-factor
        if len(row) < len(header):
            row.append(avg_b_factor)
        else:
            row[header.index("Average_B-Factor")] = avg_b_factor
        clusters_with_b_factors.append(row)

# Write the updated cluster data back to 'residue_clusters.csv'
with open('residue_clusters.csv', mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)
    writer.writerows(clusters_with_b_factors)

In [20]:
# Charge scale for residues (basic, acidic, or neutral)
charge_scale = {
    'A': 0, 'R': 1, 'N': 0, 'D': -1, 'C': 0,
    'Q': 0, 'E': -1, 'G': 0, 'H': 1, 'I': 0,
    'L': 0, 'K': 1, 'M': 0, 'F': 0, 'P': 0,
    'S': 0, 'T': 0, 'W': 0, 'Y': 0, 'V': 0
}

# Update the CSV to add Charge column
with open('asa_rsa_output.csv', mode='r') as infile:
    reader = csv.reader(infile)
    header = next(reader)
    rows = list(reader)

# Check if the header already contains "Charge"
if "Charge" not in header:
    header.append("Charge")

# Add or update charge values for each row
for row in rows:
    try:
        residue_info = row[0]  # Format is "position:amino"
        _, amino = residue_info.split(':')
        # Assign charge based on the amino acid
        charge = charge_scale.get(amino, 0)
        # If the row already has the charge column, update it, else append
        if len(row) < len(header):
            row.append(charge)
        else:
            row[header.index("Charge")] = charge
    except Exception as e:
        print(f"Error processing row {row}: {e}")

# Write the updated data back to 'asa_rsa_output.csv'
with open('asa_rsa_output.csv', mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)  # Write the updated header
    writer.writerows(rows)  # Write all rows with updated charge values